# Data Wrangling
In this notebook we'll obtain all of our data, arrange it how we want, and store it in the proper directories.

In [1]:
%run setup.ipynb
data_dir = Path('../data')
output_dir = Path('../output')

tg_nc_dir = data_dir / 'tide_gauge_nc'

# Check if the directories exist, if not create them
for d in [data_dir, output_dir, tg_nc_dir]:
    if not d.exists():
        d.mkdir()
        

## Retrieve Tide Gauge Data

We are interested in getting tide gauge and alitmetry data for the Hawaiian Islands (and surrounds) for 1993 through 2022.
Let's first establish where the tide gauges are by looking at the tide gauge dataset. We'll retrieve tide gauge data from the UHSLC (University of Hawaii Sea Level Center) fast-delivery dataset {cite:t}``. The fast-delivery data are released within 1-2 months of data collection and are subject only to basic quality control. 

We'll be retrieving the hourly data for our station group at from UHSLC, and saving this to our data directory so we don't have to download again.

In [2]:
# hawaii stations are: 
stationdict = {
    'Hilo': '060',
    'Kawaihae': '552',
    'Kahului': '059',
    'Mokuoloe': '061',
    'Honolulu': '057',
    'Nawiliwili': '058',
    'Johnston Island': '052',
    'Midway Island': '050',
    'Kaumalapau': '548',
    'Barbers Point': '547',
    'French Frigate Shoals': '014',
}
stationdict.values()

station_group = 'Hawaiian Islands'

glue('station_group', station_group)

'Hawaiian Islands'

````{margin}
```{note}
What about research quality data (RQD)? 
RQD undergo thorough and time-consuming QC, and are usually released 1-2 years after data is received. 
```
````

In [3]:
url = "https://uhslc.soest.hawaii.edu/data/netcdf/fast/hourly/" 
uhslc_ids = list(stationdict.values())

for uhslc_id in uhslc_ids:
    fname = f'h{uhslc_id}.nc' # h for hourly, d for daily

    path = os.path.join(data_dir, 'tide_gauge_nc',fname)

    if not os.path.exists(path):
        urlretrieve(os.path.join(url, fname), path) 
        print(f'Downloading {fname} from {url} to {path}')

Now we merge all the datasets. This can take a while.

In [4]:
data_dir

PosixPath('../data')

```{caution}
In the following section I remove the trailing zero from the record-id of each tide gauge. This will propagate through everything!! 
```

In [5]:
# Load the data

import glob        
# Get a list of all .nc files in the data directory
files = glob.glob(os.path.join(data_dir,'tide_gauge_nc','h*.nc'))

# Open the datasets
datasets = [xr.open_dataset(file) for file in files]

#merge in batches of 2 to avoid memory issues
batch_size = 2
merged_datasets = []

for i in range(0, len(datasets), batch_size):
    batch = datasets[i:i+batch_size]
    merged_batch = xr.merge(batch)
    merged_datasets.append(merged_batch)

#merge the merged datasets
rsl = xr.merge(merged_datasets)

# convert byte strings to normal strings
rsl['station_name'] = rsl['station_name'].astype(str)
rsl['station_country'] = rsl['station_country'].astype(str)
rsl['ssc_id'] = rsl['ssc_id'].astype(str)

# remove the trailing zero from each record_id
rsl['record_id'] =(rsl['record_id']/10).astype(int)
rsl

<xarray.Dataset> Size: 55MB
Dimensions:               (time: 1048238, record_id: 11)
Coordinates:
  * time                  (time) datetime64[ns] 8MB 1905-01-01T10:00:00.02880...
  * record_id             (record_id) int64 88B 14 50 52 57 ... 61 547 548 552
Data variables:
    sea_level             (record_id, time) float32 46MB nan nan ... 1.651e+03
    lat                   (record_id) float32 44B 23.87 28.22 ... 20.78 20.03
    lon                   (record_id) float32 44B 193.7 182.6 ... 203.0 204.2
    station_name          (record_id) <U17 748B 'French Frigate' ... 'Kawaihae'
    station_country       (record_id) <U30 1kB 'United States of America (the...
    station_country_code  (record_id) float32 44B 840.0 840.0 ... 840.0 840.0
    uhslc_id              (record_id) float32 44B 14.0 50.0 52.0 ... 548.0 552.0
    gloss_id              (record_id) float32 44B 107.0 106.0 109.0 ... nan nan
    ssc_id                (record_id) <U4 176B 'fren' 'midx' ... 'kaum' 'kawa'
    last_rq_date          (record_id) datetime64[ns] 88B 2018-10-03T22:59:59....
Attributes:
    title:                  UHSLC Fast Delivery Tide Gauge Data (hourly)
    ncei_template_version:  NCEI_NetCDF_TimeSeries_Orthogonal_Template_v2.0
    featureType:            timeSeries
    Conventions:            CF-1.6, ACDD-1.3
    date_created:           2024-08-19T14:27:33Z
    publisher_name:         University of Hawaii Sea Level Center (UHSLC)
    publisher_email:        philiprt@hawaii.edu, markm@soest.hawaii.edu
    publisher_url:          http://uhslc.soest.hawaii.edu
    summary:                The UHSLC assembles and distributes the Fast Deli...
    processing_level:       Fast Delivery (FD) data undergo a level 1 quality...
    acknowledgment:         The UHSLC Fast Delivery database is supported by ...

In [6]:
def get_MHHW_uhslc_datums(id, datumname): 
    
    url = 'https://uhslc.soest.hawaii.edu/stations/TIDES_DATUMS/fd/LST/fd'+f'{int(id):03}'+'/datumTable_'+f'{int(id):03}'+'_m.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    table = table.find_all('tr')[1:] # skip header row
    table = '<table>' + ''.join([str(x) for x in table]) # add back the table tag
    table_io = io.StringIO(str(table))
    datumtable = pd.read_html(table_io)[0]
    datum = datumtable[datumtable['Datum'] == datumname]['Value'].values[0]
    # ensure datum is a float
    datum = float(datum)
    return datum

```{note}
Should make something to include all datums here as well.
```

In [7]:
# add MHHW to the dataset
rsl['MHHW'] = xr.DataArray([1000*get_MHHW_uhslc_datums(id, 'MHHW') for id in rsl['uhslc_id'].values], dims='record_id', coords={'record_id': rsl['record_id']})

rsl['MHHW'].attrs['units'] = 'mm'
rsl['MHHW'].attrs['long_name'] = 'Mean Higher High Water, rel. to station datum'

glue('datumname', 'MHHW')

# add MSL to the dataset
rsl['MSL'] = xr.DataArray([1000*get_MHHW_uhslc_datums(id, 'MSL') for id in rsl['uhslc_id'].values], dims='record_id', coords={'record_id': rsl['record_id']})

rsl['MSL'].attrs['units'] = 'mm'
rsl['MSL'].attrs['long_name'] = 'Mean Sea Level, rel. to station datum'

glue('datumname', 'MSL')

'MHHW'

'MSL'

In [8]:
rsl

<xarray.Dataset> Size: 55MB
Dimensions:               (time: 1048238, record_id: 11)
Coordinates:
  * time                  (time) datetime64[ns] 8MB 1905-01-01T10:00:00.02880...
  * record_id             (record_id) int64 88B 14 50 52 57 ... 61 547 548 552
Data variables:
    sea_level             (record_id, time) float32 46MB nan nan ... 1.651e+03
    lat                   (record_id) float32 44B 23.87 28.22 ... 20.78 20.03
    lon                   (record_id) float32 44B 193.7 182.6 ... 203.0 204.2
    station_name          (record_id) <U17 748B 'French Frigate' ... 'Kawaihae'
    station_country       (record_id) <U30 1kB 'United States of America (the...
    station_country_code  (record_id) float32 44B 840.0 840.0 ... 840.0 840.0
    uhslc_id              (record_id) float32 44B 14.0 50.0 52.0 ... 548.0 552.0
    gloss_id              (record_id) float32 44B 107.0 106.0 109.0 ... nan nan
    ssc_id                (record_id) <U4 176B 'fren' 'midx' ... 'kaum' 'kawa'
    last_rq_date          (record_id) datetime64[ns] 88B 2018-10-03T22:59:59....
    MHHW                  (record_id) float64 88B 997.0 1.204e+03 ... 1.406e+03
    MSL                   (record_id) float64 88B 777.0 1.02e+03 ... 1.053e+03
Attributes:
    title:                  UHSLC Fast Delivery Tide Gauge Data (hourly)
    ncei_template_version:  NCEI_NetCDF_TimeSeries_Orthogonal_Template_v2.0
    featureType:            timeSeries
    Conventions:            CF-1.6, ACDD-1.3
    date_created:           2024-08-19T14:27:33Z
    publisher_name:         University of Hawaii Sea Level Center (UHSLC)
    publisher_email:        philiprt@hawaii.edu, markm@soest.hawaii.edu
    publisher_url:          http://uhslc.soest.hawaii.edu
    summary:                The UHSLC assembles and distributes the Fast Deli...
    processing_level:       Fast Delivery (FD) data undergo a level 1 quality...
    acknowledgment:         The UHSLC Fast Delivery database is supported by ...

In [9]:
#ensure rsl is netcdf4 format
rsl.to_netcdf(data_dir / 'rsl_hawaii.nc', format='NETCDF4_CLASSIC')

PermissionError: [Errno 13] Permission denied: '/Users/jfiedler/Documents/Repositories/SL_Hawaii/SL_Hawaii/data/rsl_hawaii.nc'

## Retrieve data from NOAA CO-OPS API


In [1]:
import json
url = "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations.json"

response = requests.get(url)

# get a list of all the stations
stations = json.loads(response.text)

#from list of stations, get NOAA stations within the bounding box of UHSLC gauges
hawaii_stations = [station for station in stations['stations'] if station['lat'] >= rsl['lat'].min()-2 and station['lat'] <= rsl['lat'].max()+2 and station['lng'] >= -(360-rsl['lon'].min())-2 and station['lng'] <= -(360-rsl['lon'].max())+2]

#make a dictionary of all stations in hawaii with station name: station id
stationdictNOAA  = {station['name']: station['id'] for station in hawaii_stations}
stationdictNOAA 


NameError: name 'requests' is not defined

In [2]:
import requests
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_data_chunk(stationID, start_date, end_date):
    # Format dates for the URL
    begin_date_str = start_date.strftime('%Y%m%d %H:%M')
    end_date_str = end_date.strftime('%Y%m%d %H:%M')
    
    # Create the URL
    url = f'https://tidesandcurrents.noaa.gov/api/datagetter?begin_date={begin_date_str}&end_date={end_date_str}&station={stationID}&datum=STND&product=water_level&units=metric&time_zone=gmt&format=json'
    
    # Request data from NOAA API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json()
            if 'data' in data:
                return data['data']
            else:
                # print(f"No 'data' key in response for {begin_date_str} to {end_date_str}: {data}")
                return []
        except ValueError as e:
            print(f"JSON decoding failed for {begin_date_str} to {end_date_str}: {e}")
            return []
    else:
        # print(f"Failed to fetch data for {begin_date_str} to {end_date_str}, status code: {response.status_code}")
        return []

def fetch_noaa_water_level_parallel(stationID, start_date, end_date):
    # Convert dates to datetime objects
    start_date = datetime.strptime(start_date, '%Y%m%d %H:%M')
    end_date = datetime.strptime(end_date, '%Y%m%d %H:%M')
    
    # List to hold all data
    all_data = []
    
    # Generate date ranges in 31-day increments
    date_ranges = []
    current_start_date = start_date
    while current_start_date < end_date:
        current_end_date = current_start_date + timedelta(days=31)
        if current_end_date > end_date:
            current_end_date = end_date
        date_ranges.append((current_start_date, current_end_date))
        current_start_date = current_end_date + timedelta(seconds=1)
    
    total_ranges = len(date_ranges)
    
    # Fetch data in parallel
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_date_range = {executor.submit(fetch_data_chunk, stationID, start, end): (start, end) for start, end in date_ranges}
        completed_ranges = 0
        for future in as_completed(future_to_date_range):
            try:
                data_chunk = future.result()
                if data_chunk:
                    all_data.extend(data_chunk)
            except Exception as e:
                start, end = future_to_date_range[future]
                # print(f"Error fetching data for {start} to {end}: {e}")  #uncomment if you want to track progress
            
            completed_ranges += 1
            # print(f"Progress: {completed_ranges}/{total_ranges} chunks completed.") #uncomment if you want to track progress
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    if not df.empty:
        df['t'] = pd.to_datetime(df['t'])
        
        # Clean data: remove rows with empty 'v' values
        df = df[df['v'].str.strip() != '']
        
        # Convert 'v' to float
        df['v'] = df['v'].astype(float)
        
        # Remove duplicates
        df = df.drop_duplicates(subset='t')
        
        # Sort by time
        df = df.sort_values(by='t')
        
        # Create time series
        sea_level_ts = pd.Series(df['v'].values, index=df['t'])
        
        # Resample to hourly data
        sea_level_ts = sea_level_ts.resample('h').interpolate()
        
        # Ensure unique timestamps for xarray
        sea_level_ts = sea_level_ts[~sea_level_ts.index.duplicated(keep='first')]
        
        # Create xarray dataset with the correct dimension
        ds = xr.Dataset({'sea_level': ('t', sea_level_ts.values)}, coords={'t': sea_level_ts.index})

        # rename t to time
        ds = ds.rename({'t': 'time'})
        
        
        return ds
    else:
        print("No data fetched.")
        return xr.Dataset()


In [3]:

start_date = '19050101 00:00'
end_date = '20231231 00:00'

for station_name, station_id in stationdictNOAA.items():
    # check if we already have the data
    if os.path.exists(data_dir / f'tide_gauge_nc/noaa_{station_id}.nc'):
        print(f"Data for {station_name} already exists, skipping.")
        continue
    ds = fetch_noaa_water_level_parallel(station_id, start_date, end_date)
    ds['station_id'] = station_id
    ds['station_name'] = station_name
    ds['station_country'] = 'USA'
    ds['lat'] = float([station['lat'] for station in hawaii_stations if station['id'] == station_id][0])
    ds['lon'] = float([station['lng'] for station in hawaii_stations if station['id'] == station_id][0])
    ds.to_netcdf(data_dir /  f'tide_gauge_nc/noaa_{station_id}.nc')

NameError: name 'stationdictNOAA' is not defined

As before, we'll combine them into one dataset.

In [ ]:
# Load the data

      
# Get a list of all noaa*.nc files in the data directory
files = glob.glob(os.path.join(data_dir,'tide_gauge_nc','noaa*.nc'))

# Open the datasets
datasets = [xr.open_dataset(file) for file in files]

# add the station_id as a coordinate
for ds in datasets:
    ds.coords['station_id'] = ds['station_id']
rsl = xr.concat(datasets, dim='station_id')

rsl['sea_level'].attrs['units'] = 'm'
rsl['sea_level'].attrs['long_name'] = 'Sea level, relative to station datum'

In [ ]:
rsl

Let's add some more metadata to this dataset, including MSL and MHHW datums for each gauge.

In [ ]:
MHHW = np.zeros(len(rsl['station_id']))
MSL = np.zeros(len(rsl['station_id']))

for i in range(len(rsl['station_id'])):
    url = 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/'+ str(rsl['station_id'][i].values) +'/datums.json?units=metric'
    
    response = requests.get(url)
    datums = json.loads(response.text)
    
    def extract_datum_value(data, datum_name):
        # Iterate through the datums list
        for datum in data.get('datums', []):
            # Check if the name matches the desired datum name
            if datum.get('name') == datum_name:
                # Return the value if found
                return datum.get('value')
        # Return None if the datum name is not found
        return None
    
    #extract the MHHW and MSL datums
    MHHW[i] = extract_datum_value(datums, 'MHHW')
    MSL[i] = extract_datum_value(datums, 'MSL')




In [ ]:
rsl['MHHW'] = xr.DataArray(MHHW, dims='station_id', coords={'station_id': rsl['station_id']})
rsl['MHHW'].attrs['units'] = 'm'
rsl['MHHW'].attrs['long_name'] = 'Mean Higher High Water, rel. to station datum'

rsl['MSL'] = xr.DataArray(MSL, dims='station_id', coords={'station_id': rsl['station_id']})
rsl['MSL'].attrs['units'] = 'm'
rsl['MSL'].attrs['long_name'] = 'Mean Sea Level, rel. to station datum'


Change the longitude to match UHSLC 360-degree convention.

In [ ]:
rsl['lon'] = rsl['lon'] + 360
rsl

In [ ]:
#save rsl to the data directory
rsl.to_netcdf(data_dir / 'rsl_hawaii_noaa.nc')

## Retrieve altimetry data 
We are using the global ocean gridded L4 [Sea Surface Heights and Derived Variables](https://doi.org/10.48670/moi-00148) from Copernicus. 

To download a subset of the global altimetry data, run get_CMEMS_data.py from this directory in a terminal with python >= 3.9 + copernicus_marine_client installed OR uncomment out the call to get_CMEMS_data and run it in this notebook. To read more about how to download the data from the Copernicus Marine Toolbox (new as of December 2023), visit https://help.marine.copernicus.eu/en/articles/7949409-copernicus-marine-toolbox-introduction. 

````{margin}
```{note}
You will need a username and password to access the CMEMS (Copernicus Marine Service) data if this is the first time running the client. To register for data access (free), visit https://data.marine.copernicus.eu/register.  
```
````

```{admonition} Large data download!
:class: warning
Getting errors on the code block below? Remember to uncomment "get_CMEMS_data()" to download. Note that if you change nothing in the function, it will download ~600 MB of data, which may take a long time!! You will only need to do this once. The dataset will be stored in the data directory you specify (which should be the same data directory we defined above).
```

In [ ]:
# get the min and max lat and lon of rsl for altimetry data retrieval
minlat = float(rsl.lat.min().values)
maxlat = float(rsl.lat.max().values)
minlon = float(rsl.lon.min().values)
maxlon = float(rsl.lon.max().values)

In [ ]:
def get_CMEMS_data(minlat, maxlat, minlon, maxlon, data_dir=data_dir):
        
    #maxlat = 15
    #minlat = 0
    #minlon = 125
    #maxlon = 140
    maxlat = maxlat+2
    minlat = minlat-2
    maxlon = maxlon+2
    minlon  = minlon-2
    
    start_date_str = "1993-01-01T00:00:00"
    end_date_str = "2023-04-30T23:59:59"
    data_dir = data_dir
    
    """
    Retrieves Copernicus Marine data for a specified region and time period.
    
    Args:
        minlon (float): Minimum longitude of the region.
        maxlon (float): Maximum longitude of the region.
        minlat (float): Minimum latitude of the region.
        maxlat (float): Maximum latitude of the region.
        start_date_str (str): Start date of the data in ISO 8601 format.
        end_date_str (str): End date of the data in ISO 8601 format.
        data_dir (str): Directory to save the retrieved data.
    
    Returns:
        str: The filename of the retrieved data.
    """
    copernicusmarine.subset(
        dataset_id="cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D",
        dataset_version="202112",
        variables=["sla"],
        minimum_longitude=minlon,
        maximum_longitude=maxlon,
        minimum_latitude=minlat,
        maximum_latitude=maxlat,
        start_datetime=start_date_str,
        end_datetime=end_date_str,
        output_directory=data_dir,
        output_filename="cmems_L4_SSH_0.25deg_" + start_date_str[0:4] + "_" + end_date_str[0:4] + ".nc"
    )
fname_cmems = 'cmems_L4_SSH_0.25deg_1993_2023.nc'

In [ ]:

# check if the file exists, if not, download it
if not os.path.exists(data_dir / fname_cmems):
    print('You will need to download the CMEMS data in a separate script')
    get_CMEMS_data(minlat, maxlat, minlon, maxlon, data_dir) #<<--- COMMENT OUT TO AVOID ACCIDENTAL DATA DOWNLOADS.
else:
    print('CMEMS data already downloaded, good to go!')

Open up the CMEMS data and take a look. We will want to make an ASL dataset similar in structure to the RSL data so that we can easily compare the two.

In [ ]:
# open the CMEMS data
ds = xr.open_dataset(data_dir / fname_cmems)

ds

In [ ]:
# Extract data for the nearest point to the tide gauge location that has data
sla = []
for lat, lon in zip(rsl['lat'].values, rsl['lon'].values):
    sla.append(ds['sla'].sel(
        longitude=lon-360, latitude=lat, method='nearest'
    ))

    #if the data is null, nan average over the nearest 4 points
    tol = 0.25
    if sla[-1].isnull().all():
        sla[-1] = ds['sla'].sel(
            longitude=slice(lon-360-tol, lon-360+tol), 
            latitude=slice(lat-tol, lat+tol)
        ).mean(dim=['latitude', 'longitude'])
        sla[-1]['latitude'] = np.mean(lat)
        sla[-1]['longitude'] = np.mean(lon)

sla = xr.concat(sla, dim='record_id')

#load rsl_hawaii as rsl
rsl = xr.open_dataset(data_dir / 'rsl_hawaii.nc')



# make sla a dataset with variables from rsl
sla = sla.to_dataset(name='sla')
sla['station_name'] = rsl['station_name']

# Creating lat_str and lon_str arrays with 'record_id' as their dimension
lat_str = [f'{np.abs(lat):.3f}\u00B0{"N" if lat > 0 else "S"}' for lat in sla.latitude.values]
lon_str = [f'{np.abs(lon):.3f}\u00B0{"E" if lon > 0 else "W"}' for lon in sla.longitude.values]  

# Convert lists to DataArrays with 'record_id' as their dimension
lat_str_da = xr.DataArray(lat_str, dims=['record_id'], coords={'record_id': rsl['record_id']})
lon_str_da = xr.DataArray(lon_str, dims=['record_id'], coords={'record_id': rsl['record_id']})

# Assign these DataArrays to the sla dataset
sla['lat_str'] = lat_str_da
sla['lon_str'] = lon_str_da

# add original data source to attributes
sla.attrs['original_data_source'] = 'CMEMS L4 SSH 0.25deg'
sla.attrs['title'] = ds.attrs['title']
sla.attrs['source_file'] = str(data_dir / fname_cmems)

# ensure latitude and longitude are coordinates associated with a location
sla = sla.set_coords(['latitude', 'longitude'])



sla

In [ ]:
#save sla to the data directory
sla.to_netcdf(data_dir / 'asl_hawaii.nc')

## Processing
### Process the tide gauge data to match CMEMS
 Now we'll convert tide gauge data into a daily record for the POR in units of meters to match the CMEMS data. 
 
 The next code block:
 - extracts tide gauge data for the period 1993-2022
 - converts it to meters
 - removes any NaN values
 - resamples the data to daily mean
 - and normalizes it relative to the 1993-2012 epoch. 
 

The resulting data is stored in the variable 'rsl_daily' with units in meters.

In [ ]:
# Next, let's establish a period of record from 1993-2022.
# establish the time period of interest
start_date = dt.datetime(1993,1,1)
end_date = dt.datetime(2022,12,31)
#
# also save them as strings, for plotting
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')



In [ ]:
# Extract the data for the period of record (POR)
tide_gauge_data_POR = rsl['sea_level'].sel(time=slice(start_date, end_date))

# Convert to meters and drop any NaN values
tide_gauge_data_meters = tide_gauge_data_POR / 1000  # Convert from mm to meters
# tide_gauge_data_clean = tide_gauge_data_meters.dropna(dim='time')

# Resample the tide gauge data to daily mean before subtracting the epoch mean
tide_gauge_daily_avg = tide_gauge_data_meters.resample(time='1D').mean()


In [ ]:
tide_gauge_daily_avg

### Normalize the tide gauge data relative to the 1993-2012 epoch

In [ ]:

epoch_start, epoch_end = start_date, '2011-12-31'
epoch_daily_avg = tide_gauge_daily_avg.sel(time=slice(epoch_start, epoch_end))
epoch_daily_mean = epoch_daily_avg.mean(dim='time')

In [ ]:
tide_gauge_data_POR

In [ ]:
# Subtract the epoch daily mean from the tide gauge daily average
rsl_daily = tide_gauge_daily_avg - epoch_daily_mean

# Set the attributes of the rsl_daily data
rsl_daily.attrs = tide_gauge_data_POR.attrs
rsl_daily.attrs['units'] = 'm'

# add lat and lon to the dataset
rsl_daily['lat'] = rsl['lat']
rsl_daily['lon'] = rsl['lon']

# add the station name and country
rsl_daily['station_name'] = rsl['station_name']

# change the variable name to sea level anomaly
rsl_daily.name = 'rsl_anomaly'

# change long name of the variable to sea level anomaly
rsl_daily.attrs['long_name'] = 'Sea Level Anomaly'
rsl_daily.attrs['epoch'] = '1993-2001'

rsl_daily

In [ ]:
g = rsl_daily.plot(x='time', col='record_id', col_wrap=3, sharey=False, sharex=True, figsize=(15, 10))

# Use g.axs to iterate over the axes in the FacetGrid
for ax, rid in zip(g.axs.flat, rsl_daily.record_id):
    # Accessing the station_name coordinate for the current record_id directly
    station_name = rsl_daily.station_name.sel(record_id=rid).item()
    ax.set_title(station_name)

plt.show()


In [ ]:
# Set tide gauge daily average relative to MHHW
rsl_daily_mhhw = tide_gauge_daily_avg - rsl['MHHW']/1000

# Set the attributes of the rsl_daily data
rsl_daily_mhhw.attrs = tide_gauge_data_POR.attrs
rsl_daily_mhhw.attrs['long_name'] = 'water level above MHHW'
rsl_daily_mhhw.attrs['units'] = 'm'
# add lat and lon to the dataset
rsl_daily_mhhw['lat'] = rsl['lat']
rsl_daily_mhhw['lon'] = rsl['lon']
# add the station name and country
rsl_daily_mhhw['station_name'] = rsl['station_name']

rsl_daily_mhhw.name = 'rsl_mhhw'
# save rsl_daily to the data directory
# rsl_daily_mhhw.to_netcdf(data_dir / 'rsl_daily_hawaii_mhhw.nc')


#combine the two datasets
rsl_daily_combined = xr.merge([rsl_daily, rsl_daily_mhhw])

rsl_daily_combined['storm_time'] = rsl_daily_combined['time']
rsl_daily_combined['storm_time'] = xr.DataArray(
    pd.to_datetime(rsl_daily_combined['storm_time'].values).to_series().apply(
        lambda x: x if x.month >= 5 else x - pd.DateOffset(years=1)
    ),
    dims=rsl_daily_combined['storm_time'].dims,
    coords=rsl_daily_combined['storm_time'].coords
)

#make storm time a coordinate
rsl_daily_combined = rsl_daily_combined.assign_coords(storm_time = rsl_daily_combined.storm_time)

# save rsl_daily_combined to the data directory
rsl_daily_combined.to_netcdf(data_dir / 'rsl_daily_hawaii.nc')

In [ ]:
rsl_daily_combined['storm_time'] = rsl_daily_combined['time']
rsl_daily_combined['storm_time'] = xr.DataArray(
    pd.to_datetime(rsl_daily_combined['storm_time'].values).to_series().apply(
        lambda x: x if x.month >= 5 else x - pd.DateOffset(years=1)
    ),
    dims=rsl_daily_combined['storm_time'].dims,
    coords=rsl_daily_combined['storm_time'].coords
)

#make storm time a coordinate
rsl_daily_combined = rsl_daily_combined.assign_coords(storm_time = rsl_daily_combined.storm_time)

# save rsl_daily_combined to the data directory
rsl_daily_combined.to_netcdf(data_dir / 'rsl_daily_hawaii.nc')

rsl_daily_combined


## Retrieve Climate Indices

In [3]:
def read_psl_url(url):
    response = requests.get(url)
    data = response.text
    # Split the data into lines
    lines = data.strip().split('\n')
    # Collect data lines until we encounter -9999
    data_lines = []
    for line in lines[1:]:  # Skip the first line if it's a header
        stripped_line = line.strip()
        if stripped_line == '':
            continue  # Skip empty lines
        # Detect possible footer lines (either starting with -99.9, -9999, or a textual footer)
        if stripped_line.startswith('-99.9') or stripped_line.startswith('-99.99') or stripped_line.startswith('-9999') or any(char.isalpha() for char in stripped_line):
            break  # Stop processing when footer is encountered
        # Add valid data lines to the list
        data_lines.append(line)
    # Combine data lines into a single string
    data_str = '\n'.join(data_lines)
    data_io = io.StringIO(data_str)
    return data_io

In [4]:
def read_psl_format(csvfile, varName):
    df = pd.read_csv(csvfile, sep='\s+', skiprows=1, skipfooter = 0,header=None, engine='python')
    CI = df.melt(id_vars=0, var_name='Month', value_name=varName)
    #Replace placeholder values with NaN
    CI.replace([-9.99, -9999, -9999.000, -999.99,-99.90,-99.99], np.nan, inplace=True)
    # Drop rows with NaN values
    CI = CI.dropna(subset=[varName])
    CI['time'] = pd.to_datetime({'year': CI[0], 'month': CI['Month'], 'day': 1})
    # Sort by 'time'
    CI = CI.sort_values('time').reset_index(drop=True)
    CI = CI[['time', varName]]

    return CI

In [5]:
def read_ncei_format(csvfile, varName):
    df = pd.read_csv(csvfile, sep='\s+', skiprows=0, skipfooter = 0,header=1, engine='python')
    ClimateIndex = df.melt(id_vars='Year', var_name='Month', value_name=varName)
    ClimateIndex['Month'] = ClimateIndex['Month'].map(month_mapping)
    # Replace placeholder values with NaN
    ClimateIndex.replace(99.99, np.nan, inplace=True)
    # Drop rows with NaN values
    ClimateIndex = ClimateIndex.dropna(subset=[varName])
    ClimateIndex['time'] = pd.to_datetime({'year': ClimateIndex['Year'], 'month': ClimateIndex['Month'], 'day': 1})
    # Sort by 'time'
    ClimateIndex = ClimateIndex.sort_values('time').reset_index(drop=True)
    ClimateIndex = ClimateIndex[['time', varName]]
    return ClimateIndex

In [6]:
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10, 'Nov':11, 'Dec':12
}

In [ ]:
# plot it to see if we got it right
fig, ax = plt.subplots(figsize=(15, 5))

# Plotting the ONI timeseries
oni['ONI'].plot(ax=ax, color='black', label='ONI', legend=True)

# Shading El Niño events
ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                where=oni['El Nino'] == 1, color='red', alpha=0.3, label='El Niño')

# Shading La Niña events
ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                where=oni['La Nina'] == 1, color='blue', alpha=0.3, label='La Niña')

# Adding labels and title
ax.set_xlabel('Year')
ax.set_ylabel('ONI')
ax.set_title('Oceanic Niño Index (ONI) and El Niño/La Niña Events')

# Formatting the date axis to only show the year
ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y'))

# Adding horizontal line at ONI = 0 for reference
ax.axhline(0, color='grey', linewidth=0.5, linestyle='--')

# Adding a legend
ax.legend(loc='upper left')

# set xlim to 1983 to 2023
ax.set_xlim(pd.Timestamp('1983-01-01'), pd.Timestamp('2023-12-31'))
ax.set_ylim(oni['ONI'].min(), oni['ONI'].max())

# Show the plot
plt.show()



In [8]:
# Function to convert fraction of the year to datetime
def convert_fraction_to_date(date_float):
    # Extract year and fraction
    year = int(date_float)
    fraction = date_float - year
    
    # Calculate the total number of days in the year (consider leap years)
    days_in_year = 366 if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)) else 365
    
    # Calculate the day of the year based on the fraction
    day_of_year = int(np.floor(fraction * days_in_year)) + 1
    
    # Create a datetime object for January 1st of the given year
    start_date = pd.to_datetime(f"{year}-01-01")
    
    # Add the day offset to get the actual date
    return start_date + pd.to_timedelta(day_of_year - 1, unit='D')

## Pacific Meridional Mode

In [9]:
def retrieve_pmm(yearStart='1950',yearEnd='2024',data_dir=data_dir):

    plottingUrlBase = 'https://psl.noaa.gov/cgi-bin/data/climateindices/corr.pl?'
    plotSpecs1 = 'tstype1=45&custname1=&custtitle1=&tstype2=0&custname2=&custtitle2=&'
    timeRange = 'year1='+ yearStart + '&year2=' +  yearEnd + '&'
    plotSpecs2 = 'itypea=0&y1=&y2=&plotstyle=0&length=&lag=&iall=0&iseas=0&mon1=0&mon2=11&anom=0&climo1_yr1=&climo1_yr2=&climo2_yr1=&climo2_yr2=&Submit=Calculate+Results'

    plottingUrl = plottingUrlBase + plotSpecs1 + timeRange + plotSpecs2

    # access this plotting url, and then look for text that says "Data file", and then extract the url for the data file
    response = requests.get(plottingUrl)
    soup = BeautifulSoup(response.text, 'html.parser')
    data_url = soup.find('a', string='Data file')['href']

    # download the data
    response = requests.get('https://psl.noaa.gov/'+data_url)
    data = response.text

    data_io = io.StringIO(data)
    pmm = pd.read_csv(data_io, sep='\s+', header=None, names=["time", "PMM"])

    # remove rows with missing values
    pmm = pmm.dropna()

    # Apply the conversion function to the 'Date' column
    pmm['time'] = pmm['time'].apply(convert_fraction_to_date)

    # put this time as closest to first day of the month
    pmm['time'] = pmm['time'] + pd.DateOffset(days=15)
    pmm['time'] = pmm['time'].dt.to_period('M').dt.to_timestamp()

    # save to the data directory without the index
    pmm.to_csv(data_dir / 'pmm.csv', index=False)
    print('PMM data retrieved and saved to data directory')
    return pmm

## PDO: Pacific Decadal Oscillation

Monthly SST anomalies in the North Pacific Ocean. 

In [10]:
def retrieve_pdo_ncei(data_dir=data_dir):
    url = 'https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.pdo.dat'
    response = requests.get(url)
    data = response.text
    data_io = io.StringIO(data)
    PDO = read_psl_format(data_io, 'PDO')
    PDO.to_csv(data_dir / 'pdo.csv', index=False)
    print('PDO data retrieved and saved to data directory')

    return PDO


## BEST: Bivariate EnSo Timeseries 

This climate index combines the atmospheric component of ENSO (SOI) with the oceanic component (Nino 3.4 SST). 

For more info, see: https://psl.noaa.gov/people/cathy.smith/best/details.html
Smith, C.A. and P. Sardeshmukh, 2000, The Effect of ENSO on the Intraseasonal Variance of Surface Temperature in Winter., International J. of Climatology, 20 1543-1557.

In [11]:
def retrieve_climateIndex(ClimateIndex, data_dir=data_dir):
    PSLurlBase = 'https://psl.noaa.gov/data/'

    # Define the URL based on the ClimateIndex
    if ClimateIndex in ['TNA', 'ONI','PNA']:
        url = PSLurlBase + 'correlation/' + ClimateIndex.lower() + '.data'
    
    elif ClimateIndex in ['AAO']:
        url = PSLurlBase + 'timeseries/month/data/' + ClimateIndex.lower() + '.data'

    elif ClimateIndex == 'AO':
        url = PSLurlBase + 'timeseries/month/data/' + ClimateIndex.lower() + '.long.data'

    elif ClimateIndex == 'DMI':
        url = 'https://psl.noaa.gov/gcos_wgsp/Timeseries/Data/dmieast.had.long.data'

    elif ClimateIndex == 'BEST':
        url = PSLurlBase + 'correlation/censo.data'
    
    elif ClimateIndex == 'PDO':
        url = 'https://psl.noaa.gov/pdo/data/pdo.timeseries.sstens.data'

    else:
        raise ValueError(f"ClimateIndex '{ClimateIndex}' is not recognized")
    
    print(f"Retrieving data from: {url}")

    # Fetch and format the data
    data_io = read_psl_url(url)
    climate_data = read_psl_format(data_io, ClimateIndex)

    # Save the data to a CSV file
    output_file = data_dir / (ClimateIndex.lower() + '.csv')
    climate_data.to_csv(output_file, index=False)
    print(f"{ClimateIndex} data retrieved and saved to {output_file}")

    return climate_data

In [ ]:
ONI = retrieve_climateIndex('ONI')
TNA = retrieve_climateIndex('TNA')
PNA = retrieve_climateIndex('PNA')
AAO = retrieve_climateIndex('AAO')
AO = retrieve_climateIndex('AO')
DMI = retrieve_climateIndex('DMI')
BEST = retrieve_climateIndex('BEST')
PMM = retrieve_pmm()
PDO = retrieve_climateIndex('PDO')

In [14]:
# List of all DataFrames and their corresponding names
CIs = {
    'BEST': BEST,
    'DMI': DMI,
    'AO': AO,
    'AAO': AAO,
    'PNA': PNA,
    'TNA': TNA,
    'ONI': ONI,
    'PDO': PDO,
    'PMM': PMM
}

# Start with the first DataFrame (BEST in this case)
merged_CI = CIs['BEST']

# Loop through the rest of the DataFrames and merge them on 'time'
for name, df in CIs.items():
    if name != 'BEST':
        merged_CI = pd.merge(merged_CI, df, on='time', how='outer')



In [15]:
# truncate to 1990 through 2023
merged_CI = merged_CI[(merged_CI['time'] >= '1990-01-01') & (merged_CI['time'] <= '2023-12-31')]
merged_CI.to_csv(data_dir / 'climate_indices.csv', index=False)


In [16]:
# get standard deviation of the climate indices
merged_CI = pd.read_csv(data_dir / 'climate_indices.csv')

# Calculate the standard deviation of each column except 'time'
std_dev = merged_CI.drop(columns='time').std()

# calculate the mean of each column except 'time'
mean = merged_CI.drop(columns='time').mean()

In [ ]:
mean

In [18]:
# normalize the climate indices such that they have a standard deviation of 1
merged_CI_norm = merged_CI.copy()
for col in merged_CI_norm.columns[1:]:
    # normalize the column by subtracting the mean and dividing by the standard deviation
    merged_CI_norm[col] = merged_CI_norm[col] - mean[col]
    merged_CI_norm[col] = merged_CI_norm[col] / std_dev[col]

merged_CI_norm['time'] = pd.to_datetime(merged_CI_norm['time'])

#save to the data directory
merged_CI_norm.to_csv(data_dir / 'climate_indices_norm.csv', index=False)

    

In [ ]:
merged_CI_norm

In [ ]:
import matplotlib.dates as mdates

indices = merged_CI_norm.columns[1:]

# Adjusting the x-axis limits to range from 1993 to 2024
fig, axs = plt.subplots(len(indices), 1, figsize=(10, 12), sharex=True)

# Loop through each index and its corresponding subplot
for i, index in enumerate(indices):
    axs[i].plot(merged_CI_norm['time'], merged_CI_norm[index], label=index)
    axs[i].set_ylabel(index)  
    axs[i].set_ylim(-3, 3)  # Set y-axis limits to
    axs[i].tick_params(axis='y', labelsize=8)  # Set smaller tick size for y-axis

# Set the x-axis label only for the last subplot
axs[-1].set_xlabel('Year')

# Set x-axis major ticks to every 5 years and limit to 1993 to 2024
axs[-1].set_xlim(pd.Timestamp('1990-01-01'), pd.Timestamp('2024-01-01'))
axs[-1].xaxis.set_major_locator(mdates.YearLocator(5))
axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Set a common title for the figure
fig.suptitle('Normalized Climate Indices over Time', fontsize=16)

# Adjust spacing between subplots, reducing vertical gaps
plt.tight_layout(rect=[0, 0, 1, 0.96], h_pad=0.5)


